In [1]:
import sys
print(sys.path)

['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\苏怡鸣\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\苏怡鸣\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\苏怡鸣\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\苏怡鸣\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\Program Files\\Python312\\Lib\\site-packages']


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from deeplearning_train import EarlyStopping,ModelSaver
from deeplearning_train import plot_learning_loss_curves,train_regression_model,evaluate_regression_model
import os



In [3]:
# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 加载加利福尼亚房价数据集
housing = fetch_california_housing()
X = housing.data
y = housing.target

# 数据拆分：训练集(60%)、验证集(20%)、测试集(20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)  # 0.25 x 0.8 = 0.2

print(f"训练集大小: {X_train.shape[0]}")
print(f"验证集大小: {X_val.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# 自定义数据集类
class HousingDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.FloatTensor(features)
        self.targets = torch.FloatTensor(targets).view(-1, 1)
        
    def __len__(self):
        return len(self.features) #返回样本数量
    
    def __getitem__(self, idx): #传入索引，返回对应索引样本的特征和目标
        return (self.features[idx],self.features[idx][-2:]), self.targets[idx]

# 创建数据集实例
train_dataset = HousingDataset(X_train_scaled, y_train)
val_dataset = HousingDataset(X_val_scaled, y_val)
test_dataset = HousingDataset(X_test_scaled, y_test)
# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

训练集大小: 13209
验证集大小: 3303
测试集大小: 4128


# 加载数据集，构建模型

In [4]:
# 定义Wide & Deep神经网络模型
class WideDeepModel(nn.Module):
    def __init__(self, input_dim=(8,2)):
        super(WideDeepModel, self).__init__()
        # Deep部分 - 两层全连接层，每层30个神经元
        self.deep = nn.Sequential(
            nn.Linear(input_dim[1], 60),
            nn.ReLU(),
            nn.Linear(60, 30),
            nn.ReLU(),
            nn.Linear(30, 15),
            nn.ReLU()
        )
        
        # Wide部分 - 直接使用原始特征
        self.wide = nn.Identity() # 表示恒等层，输入直接作为输出
        
        # 输出层 - 将Wide和Deep部分的输出拼接后，通过一个神经元输出
        self.output = nn.Linear(15 + input_dim[0], 1)
        
    def forward(self, x_wide,x_deep):
        # Deep路径  deep shape是[batch_size, 30]
        deep = self.deep(x_deep)
        
        # Wide路径  直接使用原始输入
        wide = self.wide(x_wide)
        
        # 连接Wide和Deep,combined shape是[batch_size, 30 + input_dim[0]]
        combined = torch.cat([wide, deep], dim=1)
        
        # 最终输出
        output = self.output(combined)
        return output


# 初始化模型、损失函数和优化器
model = WideDeepModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# 打印模型结构
print(model)


WideDeepModel(
  (deep): Sequential(
    (0): Linear(in_features=2, out_features=60, bias=True)
    (1): ReLU()
    (2): Linear(in_features=60, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=15, bias=True)
    (5): ReLU()
  )
  (wide): Identity()
  (output): Linear(in_features=23, out_features=1, bias=True)
)


In [5]:
# 训练模型
# 初始化早停和模型保存对象
early_stopping = EarlyStopping(patience=10)
model_saver = ModelSaver(save_dir='model_weights')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device='cpu'
model = model.to(device)
model, record_dict = train_regression_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=100,
    print_every=10,
    eval_step=500,
    early_stopping=early_stopping,
    model_saver=model_saver,
    device=device
)



train progress:   0%|          | 0/41300 [00:00<?, ?it/s]

早停触发! 最佳验证准确率(如果是回归，这里是损失): -0.3642
早停: 已有10轮验证损失没有改善！


In [6]:
#画图
plot_learning_loss_curves(record_dict,sample_step=200)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device='cpu'
test_loss=evaluate_regression_model(model,test_loader,device,criterion)
print(f"测试集上的损失为{test_loss:.4f}")

测试集上的损失为0.3464
